# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7feabf5d1e50>
├── 'a' --> tensor([[ 1.5064, -0.2829,  0.6767],
│                   [-0.4096,  0.6521, -1.0227]])
└── 'x' --> <FastTreeValue 0x7feabf5d1d90>
    └── 'c' --> tensor([[ 0.7473, -2.8169, -0.4367, -0.7934],
                        [-0.4365,  0.1107,  0.5621, -1.7494],
                        [ 1.1140,  0.1810, -0.0618,  0.1971]])

In [4]:
t.a

tensor([[ 1.5064, -0.2829,  0.6767],
        [-0.4096,  0.6521, -1.0227]])

In [5]:
%timeit t.a

64.5 ns ± 0.121 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7feabf5d1e50>
├── 'a' --> tensor([[-0.6853,  0.6401,  0.4658],
│                   [-0.8924, -2.1941, -1.1128]])
└── 'x' --> <FastTreeValue 0x7feabf5d1d90>
    └── 'c' --> tensor([[ 0.7473, -2.8169, -0.4367, -0.7934],
                        [-0.4365,  0.1107,  0.5621, -1.7494],
                        [ 1.1140,  0.1810, -0.0618,  0.1971]])

In [7]:
%timeit t.a = new_value

68.1 ns ± 0.0699 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[ 1.5064, -0.2829,  0.6767],
               [-0.4096,  0.6521, -1.0227]]),
    x: Batch(
           c: tensor([[ 0.7473, -2.8169, -0.4367, -0.7934],
                      [-0.4365,  0.1107,  0.5621, -1.7494],
                      [ 1.1140,  0.1810, -0.0618,  0.1971]]),
       ),
)

In [10]:
b.a

tensor([[ 1.5064, -0.2829,  0.6767],
        [-0.4096,  0.6521, -1.0227]])

In [11]:
%timeit b.a

61.8 ns ± 0.0288 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[-0.7038,  2.0007,  0.9016],
               [-0.7546,  0.2639, -0.0226]]),
    x: Batch(
           c: tensor([[ 0.7473, -2.8169, -0.4367, -0.7934],
                      [-0.4365,  0.1107,  0.5621, -1.7494],
                      [ 1.1140,  0.1810, -0.0618,  0.1971]]),
       ),
)

In [13]:
%timeit b.a = new_value

490 ns ± 0.119 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

810 ns ± 0.147 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

10.7 µs ± 18 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

143 µs ± 561 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

142 µs ± 703 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7feabc4e3be0>
├── 'a' --> tensor([[[ 1.5064, -0.2829,  0.6767],
│                    [-0.4096,  0.6521, -1.0227]],
│           
│                   [[ 1.5064, -0.2829,  0.6767],
│                    [-0.4096,  0.6521, -1.0227]],
│           
│                   [[ 1.5064, -0.2829,  0.6767],
│                    [-0.4096,  0.6521, -1.0227]],
│           
│                   [[ 1.5064, -0.2829,  0.6767],
│                    [-0.4096,  0.6521, -1.0227]],
│           
│                   [[ 1.5064, -0.2829,  0.6767],
│                    [-0.4096,  0.6521, -1.0227]],
│           
│                   [[ 1.5064, -0.2829,  0.6767],
│                    [-0.4096,  0.6521, -1.0227]],
│           
│                   [[ 1.5064, -0.2829,  0.6767],
│                    [-0.4096,  0.6521, -1.0227]],
│           
│                   [[ 1.5064, -0.2829,  0.6767],
│                    [-0.4096,  0.6521, -1.0227]]])
└── 'x' --> <FastTreeValue 0x7feab41e85b0>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

33.1 µs ± 99.9 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7fe9e6ce8130>
├── 'a' --> tensor([[ 1.5064, -0.2829,  0.6767],
│                   [-0.4096,  0.6521, -1.0227],
│                   [ 1.5064, -0.2829,  0.6767],
│                   [-0.4096,  0.6521, -1.0227],
│                   [ 1.5064, -0.2829,  0.6767],
│                   [-0.4096,  0.6521, -1.0227],
│                   [ 1.5064, -0.2829,  0.6767],
│                   [-0.4096,  0.6521, -1.0227],
│                   [ 1.5064, -0.2829,  0.6767],
│                   [-0.4096,  0.6521, -1.0227],
│                   [ 1.5064, -0.2829,  0.6767],
│                   [-0.4096,  0.6521, -1.0227],
│                   [ 1.5064, -0.2829,  0.6767],
│                   [-0.4096,  0.6521, -1.0227],
│                   [ 1.5064, -0.2829,  0.6767],
│                   [-0.4096,  0.6521, -1.0227]])
└── 'x' --> <FastTreeValue 0x7feabc4a0be0>
    └── 'c' --> tensor([[ 0.7473, -2.8169, -0.4367, -0.7934],
                        [-0.4365,  0.1107,  0.5621, -1.7494],
                 

In [23]:
%timeit t_cat(trees)

30.5 µs ± 39.6 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

66.1 µs ± 86.7 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    x: Batch(
           c: tensor([[[ 0.7473, -2.8169, -0.4367, -0.7934],
                       [-0.4365,  0.1107,  0.5621, -1.7494],
                       [ 1.1140,  0.1810, -0.0618,  0.1971]],
              
                      [[ 0.7473, -2.8169, -0.4367, -0.7934],
                       [-0.4365,  0.1107,  0.5621, -1.7494],
                       [ 1.1140,  0.1810, -0.0618,  0.1971]],
              
                      [[ 0.7473, -2.8169, -0.4367, -0.7934],
                       [-0.4365,  0.1107,  0.5621, -1.7494],
                       [ 1.1140,  0.1810, -0.0618,  0.1971]],
              
                      [[ 0.7473, -2.8169, -0.4367, -0.7934],
                       [-0.4365,  0.1107,  0.5621, -1.7494],
                       [ 1.1140,  0.1810, -0.0618,  0.1971]],
              
                      [[ 0.7473, -2.8169, -0.4367, -0.7934],
                       [-0.4365,  0.1107,  0.5621, -1.7494],
                       [ 1.1140,  0.1810, -0.0618,  0.1971]],

In [26]:
%timeit Batch.stack(batches)

78.5 µs ± 191 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    x: Batch(
           c: tensor([[ 0.7473, -2.8169, -0.4367, -0.7934],
                      [-0.4365,  0.1107,  0.5621, -1.7494],
                      [ 1.1140,  0.1810, -0.0618,  0.1971],
                      [ 0.7473, -2.8169, -0.4367, -0.7934],
                      [-0.4365,  0.1107,  0.5621, -1.7494],
                      [ 1.1140,  0.1810, -0.0618,  0.1971],
                      [ 0.7473, -2.8169, -0.4367, -0.7934],
                      [-0.4365,  0.1107,  0.5621, -1.7494],
                      [ 1.1140,  0.1810, -0.0618,  0.1971],
                      [ 0.7473, -2.8169, -0.4367, -0.7934],
                      [-0.4365,  0.1107,  0.5621, -1.7494],
                      [ 1.1140,  0.1810, -0.0618,  0.1971],
                      [ 0.7473, -2.8169, -0.4367, -0.7934],
                      [-0.4365,  0.1107,  0.5621, -1.7494],
                      [ 1.1140,  0.1810, -0.0618,  0.1971],
                      [ 0.7473, -2.8169, -0.4367, -0.7934],
                   

In [28]:
%timeit Batch.cat(batches)

146 µs ± 556 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

338 µs ± 1.79 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
